# Sprint14 ディープラーニングフレームワーク2

## このSprintについて

### Sprintの目的
- フレームワークのコードを読めるようにする
- フレームワークを習得し続けられるようになる
- 理論を知っている範囲をフレームワークで動かす

### どのように学ぶか¶
前半はTensorFlowのExampleを動かします。後半ではKerasのコードを書いていきます。

## 公式Example
深層学習フレームワークには公式に様々なモデルのExampleコードが公開されています。

### 【問題1】公式Exampleを分担して実行
TensorFLowの公式Exampleを分担して実行してください。

以下の中から1人ひとつ選び実行し、その結果を簡単に発表してください。

#### research

定番のモデルから最新のモデルまで多様なコードが公開されています。

[models/research at master · tensorflow/models](https://github.com/tensorflow/models/tree/master/research)

#### tutorials

TensorFLowのチュートリアルとして用意された簡単なモデルが含まれています。

[models/tutorials at master · tensorflow/models](https://github.com/tensorflow/models/tree/master/tutorials)

### 【解答】
#### [How to train Boosted Trees models in TensorFlow](https://www.tensorflow.org/tutorials/estimators/boosted_trees)を参照
- TensorFlowでBoosted Treesモデルをトレーニングする方法のチュートリアル
- タイタニックデータセットを、Boosted Treesモデルを用いて生存者予測を行う
- ロジスティック回帰モデルとBoosted Treesモデルの推定結果を比較し後者の方が高い指標を出したことを確認

## 異なるフレームワークへの書き換え
Sprint14で作成した4種類のデータセットを扱うTensorFLowのコードを異なるフレームワークに変更していきます。

- Iris（Iris-versicolorとIris-virginicaのみの2値分類）
- Iris（3種類全ての目的変数を使用して多値分類）
- House Prices
- MNIST

### Kerasへの書き換え
KerasはTensorFLowに含まれるtf.kerasモジュールを使用してください。

KerasにはSequentialモデルかFunctional APIかなど書き方に種類がありますが、これは指定しません。

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf

### 【問題2】Iris（2値分類）をKerasで学習
TensorFlowによるIrisデータセットに対する2値分類をKerasに書き換えてください。

In [2]:
# データセットの読み込み
dataset_path ="Iris.csv"
df = pd.read_csv(dataset_path)
# データフレームから条件抽出
df = df[(df["Species"] == "Iris-versicolor")|(df["Species"] == "Iris-virginica")]
y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
y = np.array(y)
X = np.array(X)
# ラベルを数値に変換
y[y=='Iris-versicolor'] = 0
y[y=='Iris-virginica'] = 1

# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

In [3]:
class GetMiniBatch:
    """
    ミニバッチを取得するイテレータ

    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      学習データ
    y : 次の形のndarray, shape (n_samples, 1)
      正解値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyの乱数のシード
    """
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]        
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]

n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 1

model = tf.keras.Sequential([
            tf.keras.layers.Dense(n_hidden1, activation = tf.nn.relu, input_shape=(4,)),
            tf.keras.layers.Dense(n_hidden2, activation = tf.nn.relu),
            tf.keras.layers.Dense(n_classes, activation = tf.sigmoid)])

model.compile(loss='binary_crossentropy',
              optimizer=tf.train.AdamOptimizer(learning_rate=0.01),
              metrics=['accuracy'])

history = model.fit(X_train, y_train,
                    batch_size=1,
                    epochs=10,
                    verbose=0,
                    validation_data=(X_val, y_val))

y_pred_proba = model.predict(X_test)[:, 0]

# 確率を0, 1に変換
y_pred = np.where(y_pred_proba >0.5, 1, 0)

print("y_pred_proba", y_pred_proba)
print("y_pred", y_pred)
print("y_test", y_test)
score = model.evaluate(X_test, y_test, verbose=0)
print('Train loss:', score[0])
print('Train accuracy:', score[1])

W1021 21:23:33.067892  5104 deprecation.py:506] From C:\Anaconda3\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W1021 21:23:33.208465  5104 deprecation.py:323] From C:\Anaconda3\lib\site-packages\tensorflow\python\ops\nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


y_pred_proba [0.00628477 0.91585267 0.00462064 0.9515404  0.566797   0.9318322
 0.02009654 0.52911377 0.9383583  0.8762684  0.82796335 0.8667389
 0.93018126 0.01177433 0.00276574 0.00339201 0.21802373 0.00183203
 0.52169394 0.0023486 ]
y_pred [0 1 0 1 1 1 0 1 1 1 1 1 1 0 0 0 0 0 1 0]
y_test [0 1 0 1 1 1 0 1 1 1 1 1 1 0 0 0 0 0 0 0]
Train loss: 0.15246181190013885
Train accuracy: 0.95


### 【問題3】Iris（多値分類）をKerasで学習
TensorFlowによるIrisデータセットに対する3値分類をKerasに書き換えてください。

In [5]:
from sklearn.preprocessing import OneHotEncoder

# データセットの読み込み
dataset_path ="Iris.csv"
df = pd.read_csv(dataset_path)
# データフレームから条件抽出
y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
y = np.array(y)
X = np.array(X)
# ラベルを数値に変換
y[y=='Iris-setosa'] = 0
y[y=='Iris-versicolor'] = 1
y[y=='Iris-virginica'] = 2
y = y.astype(np.int)[:, np.newaxis]

enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
y_one_hot = enc.fit_transform(y[:])

# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y_one_hot, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 3

# 計算グラフに渡す引数の形を決める
model = tf.keras.Sequential([
            tf.keras.layers.Dense(n_hidden1, activation = tf.nn.relu, input_shape=(4,)),
            tf.keras.layers.Dense(n_hidden2, activation = tf.nn.relu),
            tf.keras.layers.Dense(n_classes, activation = tf.nn.softmax)])

model.compile(loss='categorical_crossentropy',
              optimizer=tf.train.AdamOptimizer(learning_rate=0.01),
              metrics=['accuracy'])

history = model.fit(X_train, y_train,
                    batch_size=1,
                    epochs=10,
                    verbose=0,
                    validation_data=(X_val, y_val))

y_pred_proba = model.predict(X_test)#[:, 0]

# 確率を0, 1に変換
y_pred = np.argmax(y_pred_proba, axis=1)

print("y_pred", y_pred)
print("y_test", np.argmax(y_test, axis=1))
score = model.evaluate(X_test, y_test, verbose=0)
print('Train loss:', score[0])
print('Train acc:', score[1])

y_pred [2 1 0 2 0 2 0 1 1 1 2 1 1 1 1 0 1 1 0 0 2 1 0 0 2 0 0 1 1 0]
y_test [2 1 0 2 0 2 0 1 1 1 2 1 1 1 1 0 1 1 0 0 2 1 0 0 2 0 0 1 1 0]
Train loss: 0.03252994641661644
Train acc: 1.0


### 【問題4】House PricesをKerasで学習
TensorFlowによるHouse Pricesデータセットに対する回帰をKerasに書き換えてください。

In [6]:
# データセットの読み込み
dataset_path ="train.csv"
df = pd.read_csv(dataset_path)
# データフレームから条件抽出
y = df["SalePrice"]
X = df.loc[:, ["GrLivArea", "YearBuilt"]]
y = np.array(y)
X = np.array(X)
# ラベルを数値に変換
y = y.astype(np.int)[:, np.newaxis]
y = np.log(y)

# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 1

# 計算グラフに渡す引数の形を決める

model = tf.keras.Sequential([
            tf.keras.layers.Dense(n_hidden1, activation = tf.nn.relu, input_shape=(2,)),
            tf.keras.layers.Dense(n_hidden2, activation = tf.nn.relu),
            tf.keras.layers.Dense(n_classes)])

model.compile(loss='mean_squared_error',
              optimizer=tf.train.AdamOptimizer(learning_rate=0.01),
              metrics=['mse'])

history = model.fit(X_train, y_train,
                    batch_size=1,
                    epochs=10,
                    verbose=0,
                    validation_data=(X_val, y_val))

y_pred = model.predict(X_test)

#print("y_pred", y_pred)
#print("y_test", y_test)
score = model.evaluate(X_test, y_test, verbose=0)
print('Train loss:', score[0])
print('Train mse:', score[1])

Train loss: 0.06368532368581589
Train mse: 0.06368532


### 【問題5】MNISTをKerasで学習
TensorFlowによるMNISTデータセットによる画像の多値分類をKerasに書き換えてください。

In [7]:
from keras.datasets import mnist

Using TensorFlow backend.


In [8]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.reshape(-1, 784)
X_test = X_test.reshape(-1, 784)

X_train = X_train.astype(np.float)
X_test = X_test.astype(np.float)
X_train /= 255
X_test /= 255

y_train = y_train.astype(np.int)[:, np.newaxis]
y_test = y_test.astype(np.int)[:, np.newaxis]

enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
y_train_one_hot = enc.fit_transform(y_train[:])
y_test_one_hot = enc.fit_transform(y_test[:])

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train_one_hot, test_size=0.2)

n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 10

# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])


model = tf.keras.Sequential([
            tf.keras.layers.Dense(n_hidden1, activation = tf.nn.relu, input_shape=(784,)),
            tf.keras.layers.Dense(n_hidden2, activation = tf.nn.relu),
            tf.keras.layers.Dense(n_classes, activation = tf.nn.softmax)])

model.compile(loss='categorical_crossentropy',
              optimizer=tf.train.AdamOptimizer(learning_rate=0.01),
              metrics=['accuracy'])

history = model.fit(X_train, y_train,
                    batch_size=1,
                    epochs=10,
                    verbose=0,
                    validation_data=(X_val, y_val))

y_pred_proba = model.predict(X_test)#[:, 0]

# 確率を0, 1に変換
y_pred = np.argmax(y_pred_proba, axis=1)

print("y_pred", y_pred)
print("y_test", np.argmax(y_test_one_hot, axis=1))
score = model.evaluate(X_test, y_test_one_hot, verbose=0)
print('Train loss:', score[0])
print('Train acc:', score[1])

KeyboardInterrupt: 

### 【問題6】（アドバンス課題）PyTorchへの書き換え
4種類の問題をPyTorchに書き換えてください。

### 【問題7】（アドバンス課題）フレームワークの比較
それぞれのフレームワークにはどのような違いがあるかをまとめてください。

#### 視点例
- 計算速度
- コードの行数・可読性
- 用意されている機能